In [ ]:
import pandas as pd
dir_hackathon = '.'
dir_data_in = f'{dir_hackathon}/Data/In'
dir_data_out = f'{dir_hackathon}/Data/Out'

Загрузка данных. Колонки "Коды ТН ВЭД ЕАЭС", "Технические регламенты" и "Группа продукции" - категориальные. Нам нужно их обработать и превратить в численные. Применим два подхода - нумерацию и разбиение на много колонок.

In [ ]:
all_data_in = pd.read_excel(f'{dir_data_in}/dataset2.xlsx', header=0)
all_data_in.columns = ['Номер продукции', 'Коды ТН ВЭД ЕАЭС', 'Технические регламенты', 'Группа продукции', 'Общее наименование продукции', 
                       'ИЛ', 'Заявитель', 'Адрес Заявителя', 'Изготовитель', 'Страна', 'Адрес изготовителя']

all_data_in.head(2)

Нам нужно обработать  колонки 'Коды ТН ВЭД ЕАЭС', 'Технические регламенты', 'Группа продукции как категориальные.  У нас есть категориальные данные двух видов. Группы продукции - самое простое, просто строка, обозначающее название группы. Обработаем штатными средствами. С двумя другими сложнее - там в каждой ячейке группы перечислены через разделитель, да ещё и с дубликатами. Их будем обрабатывать отдельно. 

In [ ]:
object_cols = ['Группа продукции', 'Технические регламенты']
object_cols_target = ['Группа продукции_ordinal', 'Технические регламенты_ordinal'	] # В такую колонку положим результат простой нумерации
all_data_in[object_cols]


Посмотрим, сколько есть различных категорий

In [ ]:
for col in object_cols:
    count = len(all_data_in[col].value_counts())
    print(f"{col} : {count}")

Ordinal encoder. Нумеруем категории. 


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
data_copy = all_data_in.copy()
all_data_in[object_cols_target] = ordinal_encoder.fit_transform(data_copy[object_cols])
all_data_in[object_cols + object_cols_target].head(10)

OneHotEncoder. Проставляем каждой группе, соотвествтвует ли ей категория. Есть сомнения, что это вообще нужно, но попробуем.


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# object_cols = ['Группа продукции', 'Технические регламенты']
object_cols = ['Группа продукции']
# Apply one-hot encoder to each column with categorical data
data_copy = all_data_in[object_cols]
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False) # можно преобразовать вид, убрав sparse=False. Будет одна колонка. 
OH_cols = pd.DataFrame(OH_encoder.fit_transform(data_copy))
print(col)
OH_cols.head(5)
col_names = [ f"{col}_{oh_col}" for oh_col in OH_cols.columns]
all_data_in[col_names] = OH_cols

Теперь код ТН ВЭД и регламенты. Опишем в виде функции преобразование, затем применим к каждой колонке

In [ ]:
object_cols = ['Коды ТН ВЭД ЕАЭС', ]
all_data_in[object_cols]

Nan - это пустые строчки, где не заданы значения. Заменим на "Not presented". 

In [ ]:
all_data_in[object_cols] = all_data_in[object_cols].fillna("Not presented") # почему-то inplace не работает
all_data_in[object_cols].tail(5) #там были NaN


In [ ]:
def make_categorical_data_numeric(df, cols, delim=";"):
    for col in cols:
        df_col = df[col]
        group = set()
        for el in df_col:
            groups_in_el = el.split(delim)
            for gr in groups_in_el:
                group.add(gr)
        new_cols = pd.DataFrame(columns=list(group))
        for el in df_col:
            row = [1 if col in el else 0 for col in group]
            new_cols.loc[len(new_cols)] = row
        df[list(group)] = new_cols
    return df

changed_data = make_categorical_data_numeric(all_data_in[object_cols], object_cols, ";") 
changed_data.head(10)
        

In [ ]:
all_data_in[object_cols].to_string()
all_data_in[object_cols].head(5)
